# 1. LWE and RLWE encryption
## 1-1. LWE encryption

In FHEW-like HE, we use both LWE and RLWE encryption.
LWE is for a ciphertext and RLWE is for the core part of bootstrapping so called *blind rotation*.

We define LWE encryption as follows
$$\textsf{LWE}_{\vec{s}, n, q}(m) = (\beta, \vec{\alpha}) \in \mathbb{Z}_q^{n+1},$$
where $\beta = m + e - \left< \vec{\alpha}, \vec{s} \right> \in \mathbb{Z}_q$, and $\vec{s} \leftarrow \chi_{key}$ is a secret key and $\vec{e} \leftarrow \chi_{err}$ is a added noise for security.
$\vec{\alpha}$ is unifromly sampled in $\mathbb{Z}_q^n$.

Here, we choose $\chi_{key}$ as binary distribution and $\chi_{err}$ as a Gaussian distribution with standard deviation $3.2$.

Let's make the encryption method. We use [pytorch](https://pytorch.org/) for easy and fast implementation.

In [1]:
import torch
import numpy as np


We use parameter sets $(n, q, \sigma) = (512, 2048, 3.2)$.

In [2]:
stddev = 3.2
n = 512
q = 2048

Following are generator of key and error.

NOTE: Those generators are not secure. You should **NOT** use them in practice.

In [3]:
def keygen(dim):
    return torch.randint(2, size = (dim,))

def errgen(stddev):
    e = torch.round(stddev*torch.randn(1))
    e = e.squeeze()
    return e.to(torch.int)

def uniform(dim, modulus):
    return torch.randint(modulus, size = (dim,))

We first generate the secret key $\vec{s}$.

In [4]:
s = keygen(n)
s

tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
        0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
        0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
        1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,

To encrypt, we need random part $\vec{\alpha}$.

In [5]:
alpha = uniform(n, q)
alpha

tensor([2046,  458, 1484,  884,  417, 1187,   70, 1243,  598,  476, 1630, 1536,
        1796,  411,  747, 1870,  718,  946, 1535,   75,  790, 1915, 1783,  700,
        1553,  191,  839, 1342,  786, 1596, 1884,  440,    3, 1184,  971,  215,
        1890,  677, 1550, 1968,   43, 1937, 1647,  588, 1385,   86,    8,   76,
        1430, 1221, 1397, 1248,  179, 1750,  310,  729, 1449,  800, 1889,  629,
         810,  440,  739,  369, 1795, 1997,  629, 1528, 1365,  866,   81,  702,
        1889, 1200,   79, 1225, 1920,  211, 1905, 1014, 1072,  797,  620,  971,
        1594,  258, 1829, 1268, 1701,  530, 1123,  279, 1697,  660, 1366, 1414,
         311,   86, 2018, 1795, 1592, 1574, 1429, 1968, 1750,  198,  985,  785,
        1254, 1870, 1643, 1626, 1037, 1772,  173,  633, 1184, 1819, 1206,  982,
        1100,   17, 1778,  879, 1158,  150,  181, 1755, 1787, 1473, 1613, 1949,
         447, 1257,  475,  459, 1710,  376,   18,  487,   71, 1459,  394, 1573,
        1795,  290,  454, 1606, 1472, 10

We calaulate $\beta = m + e - \left< \vec{\alpha}, \vec{s} \right>$ for encryption.

Let the message we are encrypting is a binary value e,g, $m = 1$ here.

In [6]:
m = 1

beta = m - torch.dot(alpha, s)
e = errgen(stddev)
beta += e

beta %= q

beta

tensor(306)

By *LWE assumption* $\beta$ should look like a random value.
Now the pair $(\beta, \vec{\alpha})$ is our ciphertext.

Let's decrypt the ciphertext above.

As $\beta = m + e - \left< \vec{\alpha}, \vec{s} \right>$, we can find $m + e = \beta + \left< \vec{\alpha}, \vec{s} \right>$.

In [7]:
m_decrypted = beta + torch.dot(alpha, s)
m_decrypted %= q
m_decrypted

tensor(1)

If you are very lucky, you might get the decrypted value.
```
>>> m_decrypted
tensor(1)
```
But, if you run the code once, you will get other value.
Note here that we get $m+e$ by decryption, *not the exact value* $m$.

To make our message safe from the error, we can multiply certain *scaling factor* to our message.

Here, let's multiply $q/4$, and encrypt/decrypt again.


In [8]:
m = 1
# multiply scaling factor q/4 
m *= q//4

beta = m - torch.dot(alpha, s)
e = errgen(stddev)
beta += e
beta %= q

m_decrypted = beta + torch.dot(alpha, s)
m_decrypted %= q

m_decrypted

tensor(513)

We got a value near $m \cdot q/4 = 512$.
Division by $q/4$ and rounding will give us original message.

In [9]:
# rescale the message
m_decrypted = m_decrypted.to(torch.float)
m_decrypted /= q/4.
m_decrypted = torch.round(m_decrypted)
m_decrypted.to(torch.int)


tensor(1, dtype=torch.int32)

Decryption is successful!


### LWE encryption function

The LWE ciphertext is a pair $(\beta, \vec{\alpha})$.

We define the encryptor as follows.

In [10]:
def encryptLWE(message, dim, modulus, key):
    alpha = uniform(dim, modulus)

    beta = message * modulus//4 - torch.dot(alpha, key)
    e = errgen(stddev)
    beta += e
    beta %= modulus

    return (beta, alpha)


ct = encryptLWE(1, n, q, s)
ct    

(tensor(740),
 tensor([ 492,  208, 1332,  531,  109, 1803,  186,  413, 1289,  558, 1651,  247,
          937,  604,  241, 1369, 1096,  564, 1334, 1842, 2027,  453, 1886,   36,
           50, 1663, 1056,  732,  597,  956,  534, 1227,  196, 1658, 1458, 1103,
         2037, 1770, 1727, 1793, 1961, 1666,  680, 1342,  727, 1326,  436, 1491,
         1825, 1130, 1636,   54,  855, 1983, 1251,  453,  622,  377, 1824,  477,
          593, 1678,  830,  219,  178, 1878, 1138,  203,  400,  721, 1214, 1665,
         1145,   41, 1101, 1345, 1664, 1892, 1854,  480,  380,  375,  619,  835,
          222, 1899,  151, 1032, 1231,  496, 1769,  660, 1116, 1189,  396, 1641,
          656, 1979,  934, 1460,  996,  452,  766, 1628, 1663, 1096,  293,  960,
         1249,  826,  296,  801, 1475, 1663, 1712,  542, 1136, 1362, 1943,  815,
          576, 1279, 1523,   55, 1329, 1907,  340, 1667,  916,  578,  949,  427,
         1445,  965, 1145, 1831,  368, 1229, 1859,  918,  546,  506, 1108, 2027,
          776,

### LWE decryption function

We can also define decryption.

In [11]:
def decryptLWE(ct, key, modulus):
    beta, alpha = ct
    m_dec = beta + torch.dot(alpha, key)
    m_dec %= modulus

    m_dec = m_dec.to(torch.float)
    m_dec /= modulus/4.
    m_dec = torch.round(m_dec)
    return m_dec.to(torch.int)

decryptLWE(ct, s, q)   

tensor(1, dtype=torch.int32)

## 1-2. RLWE encryption

RLWE (a.k.a Ring-LWE) is a ring variant of LWE. The encrypted message in RLWE is a polynomial (with $N$ integer coefficients) rather than an integer.

RLWE is more efficient, and easily define multiplication.

In FHEW-like HE, we use the fact the the encrypted message is a *polynomial* and do computation on the exponent of $X$.

### Integer ring

As we use ring structure for RLWE, we define some important notation here. Please find detailed explanation in other papers. 

#### Polynomial

We define set of polynomial $\mathbb{Z}[X] = \{\sum_i a_i X^i : a_i \in \mathbb{Z}\}$.
Polynomial is denoted by bold characters e.g., $\boldsymbol{a}$.

#### Polynomial ring

Usually, we set $N$ a power-of-two for efficiency.
$\mathcal{R} = \mathbb{Z}[X]/\left< X^N+1 \right>$ denotes polynomial ring, and $\phi_{2N}(X) = X^N+1$ is also called as a $2N$-th primitive polynomial.

$\mathcal{R}$ is a set of polynomial whose degree is less than $N$, and we can define addition and multiplication here.
 
By multiplying two polynomials, the degree of product can be greater than or equal to $N$, in that case, we divide the product by $X^N + 1$ and use only the remainder.

We define $\mathcal{R}_Q = \mathcal{R}/Q\mathcal{R}$ as integer ring whose coefficients are modulus $Q$.

$Q$ is selected as a prime number and $Q \equiv 1 \pmod{2N}$ for efficiency of NTT.

However, for bervity, we just use 2**27.


In [12]:
N = 2**10
Q = 2**27


#### implementation of integer polynomial

We use torch tensor to represent a polynomial in $\mathcal{R}_Q$ of length $N$, where its i-th element is coefficient of $X^i$


In [13]:
# random polynomial
a = uniform(N, Q)
a

tensor([35215391, 18094837, 47150314,  ..., 51357027, 97790261, 85164842])

### RLWE encryption and decryption
We define RLWE encryption as follows
$$\textsf{RLWE}_{N,Q,\boldsymbol{z}} = (\boldsymbol{b}, \boldsymbol{a}) \in \mathcal{R}_Q^2,$$
where $\boldsymbol{b} = \boldsymbol{m} + \boldsymbol{e} - \boldsymbol{a} \cdot \boldsymbol{z}$.
Here, $\boldsymbol{a}$ is sampled unifromly from $\mathcal{R}_Q$ and $\boldsymbol{z} \rightarrow \chi_{key}$ is key, and $\boldsymbol{e} \rightarrow \chi_{err}$ is added noise for security, similar to LWE.

#### Why RLWE is more efficient than LWE?
In HE, there are many reasons why RLWE is more efficient than LWE, but you can find the following if you see the equation carefully.

An LWE ciphertext is composed of $n+1$ integers but encrypts only one value.
However, RLWE ciphertext is compose of $2$ polynomials (=$2N$ integers), but encrypts $1$ polynomial (=$N$ integer coefficients).

#### Let's code
We need a secret key $\boldsymbol{z}$.


In [14]:
z = keygen(N)
z

tensor([1, 0, 1,  ..., 1, 0, 1])

Let our message $(1,0,0,0, ....)$.

In [15]:
m = torch.zeros(N).to(torch.int)
m[0] = 1
m

tensor([1, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

For encryption, we need $\boldsymbol{a}$ and $\boldsymbol{e}$.

In [16]:
a = uniform(N, Q)
a


tensor([108202011,  80816390, 119055355,  ...,  60346144,  90332141,
        127142132])

In [17]:
def errpolygen(dim, stddev):
    e = torch.round(stddev*torch.randn(dim))
    e = e.squeeze()
    return e.to(torch.int)

e = errpolygen(N, stddev)
e

tensor([ 0,  1,  0,  ...,  4, -1,  1], dtype=torch.int32)

We need multiplication between polynomials $\boldsymbol{a}$ and $\boldsymbol{z}$ for encryption.

Let the $i$-th coefficient of $\boldsymbol{a} \cdot \boldsymbol{z}$ be $(\boldsymbol{a} \cdot \boldsymbol{z})_i$.

Then, we have $ (\boldsymbol{a} \cdot \boldsymbol{z})_i = \sum_{j \le i} a_j \cdot z_{i-j} + \sum_{j > i} - a_j \cdot z_{N + i-j} $.

The negative terms from the fact that $X^N = -1$ as we divide polynomials by $X^N+1$.

In [18]:
def polymult(a, b, dim, modulus):
    res = torch.zeros(dim).to(torch.int)
    for i in range(dim):
        for j in range(dim):
            if i >= j:
                res[i] += a[j]*b[i-j]
                res[i] %= modulus
            else:
                res[i] += modulus - a[j]*b[i-j] # Q - x mod Q = -x
                res[i] %= modulus

    res %= modulus
    return res

Now, we can make encrypt $\boldsymbol{m}$.

In LWE we used scaling factor $q/4$, but let me use 256 here for fun (large enough).

In [19]:
b = e - polymult(a, z, N, Q)
b += (m * 2**8)
b %= Q
b

tensor([103170463, 102521568,  24789522,  ..., 100989651,  17044564,
         69928782], dtype=torch.int32)

Note polymult is **super slow** - in my M2-pro, it took 10.1s.

The time complexity is $O(n^2)$

We will handle this later (using NTT).

The pair $(\boldsymbol{b}, \boldsymbol{a})$ is our RLWE encryption.

In [20]:
ct = (b, a)
ct

(tensor([103170463, 102521568,  24789522,  ..., 100989651,  17044564,
          69928782], dtype=torch.int32),
 tensor([108202011,  80816390, 119055355,  ...,  60346144,  90332141,
         127142132]))

We can decrypt the ciphertext using

$ \boldsymbol{m} + \boldsymbol{e}  = \boldsymbol{b} + \boldsymbol{a}\cdot \boldsymbol{z}$.

In [21]:
b, a = ct

m_decrypted = b + polymult(a, z, N, Q)
m_decrypted %= Q
m_decrypted

tensor([      256,         1,         0,  ...,         4, 134217727,
                1], dtype=torch.int32)

Values less than 0 will look like very big, e.g. -1 = 2**27 -1. 
Normalize it.

In [22]:
for i in range(N):
    if m_decrypted[i] > Q//2:
        m_decrypted[i] -= Q

m_decrypted

tensor([256,   1,   0,  ...,   4,  -1,   1], dtype=torch.int32)

In [23]:
m_decrypted = m_decrypted.to(torch.float) / float(2**8)
torch.round(m_decrypted)
m_decrypted = torch.round(m_decrypted).to(torch.int)

m_decrypted

tensor([1, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

We learned LWE/RLWE encryption and decryption in this chapter!